In [15]:
# !sudo apt update

In [16]:
# !sudo apt install -y libopenblas-dev pkg-config

In [33]:
!pip install emblaze trimap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 20.1 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp313-cp313-linux_x86_64.whl size=82757 sha256=9f877f9750106d1bc608f749e1147b93bd22d82b5292868756f2499daa767bf8
  Stored in directory: /home/jovyan/.cache/pip/wheels/fc/4e/87/f1f957e7382aa370452cff98276f51686924d5415e449800ed
Successfully built annoy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [trimap]


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import emblaze
import trimap
from emblaze.utils import Field, ProjectionTechnique
from collections import defaultdict, namedtuple

/opt/conda/lib/python3.13/site-packages/trimap/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [19]:
def load_word2vec_format(fname):
    """Load embeddings from word2vec format file.

    Returns a dictionary with 'vectors' (numpy array) and 'index_to_key' (list of keys).
    """
    vectors = []
    index_to_key = []

    with open(fname, 'r') as f:
        # Read header: num_vectors embedding_dim
        header = f.readline().strip().split()
        num_vectors = int(header[0])
        embedding_dim = int(header[1])

        # Read each vector
        for line in f:
            parts = line.strip().split()
            key = parts[0]  # The index/key
            vector = list(map(float, parts[1:]))

            index_to_key.append(key)
            vectors.append(vector)

    return {
        'vectors': np.array(vectors),
        'index_to_key': index_to_key
    }

fname = '../data/pymde_3d_10n_bigger.w2v'
wv = load_word2vec_format(fname)

In [20]:
# load mapping from embedding index to score ID
mapping_fname = f'{fname}.csv'
mapping_df = pd.read_csv(mapping_fname)
score_id_by_ix = mapping_df.set_index('index')['score_id'].to_dict()

In [21]:
my_score_ids = set(pd.read_csv('../data/my_score_ids.csv')['score'].values)
blankz_score_ids = set(pd.read_csv('../data/blankz_score_ids.csv')['score'].values)
big_tomas_score_ids = set(pd.read_csv('../data/big_tomas_score_ids.csv')['score'].values)
peblohh_score_ids = set(pd.read_csv('../data/peblohh_score_ids.csv')['score'].values)

In [22]:
score_metadata = pd.read_csv(f'../data/score_metadata.csv')

score_metadata_by_id = {}
for row in score_metadata.itertuples():
    score_metadata_by_id[row.score_id] = row

next(iter(score_metadata_by_id.values()))

Pandas(Index=0, score_id='1000052_DTHR', avg_pp=97.87299, num_users=3)

In [23]:
len(score_metadata)

207687

In [24]:
# load beatmap metadata
beatmaps_df = pd.read_parquet('../data/beatmaps.parquet')
# beatmaps_df.head()

beatmaps_by_id = {}
for row in beatmaps_df.itertuples():
    beatmaps_by_id[row.beatmap_id] = row

next(iter(beatmaps_by_id.values()))

Pandas(Index=0, id=1.0, beatmapset_id=40071, beatmap_id=127363, approved=1, approved_date=Timestamp('2012-01-15 21:13:09'), last_update=Timestamp('2012-01-15 21:07:48'), total_length=233, hit_length=193, version='Posthumous', artist='DJ Okawari', title='Luv Letter', creator='nold_1702', bpm=95, source='', difficultyrating=5.31721, diff_size=4, diff_overall=8, diff_approach=9, diff_drain=6, mode=0)

In [32]:
positions = wv['vectors']
names = [score_id_by_ix[index] for index in range(len(wv['vectors']))]
# colors = [int(score_id_by_ix[index].split('_')[0]) for index in range(len(wv['vectors']))]
# score ids are in the format `{id}_{mods}`, and we color by whether the score has DT, HT, or neither
# colors = [int('DT' in score_id_by_ix[index]) + 2 * int('HR' in score_id_by_ix[index]) for index in range(len(wv['vectors']))]
# colors = [1 if score_id_by_ix[index] in my_score_ids else 0 for index in range(len(wv['vectors']))]
colors = [max(80,min(score_metadata_by_id[score_id_by_ix[index]].avg_pp,1300)) for index in range(len(wv['vectors']))]
# colors = [beatmaps_by_id[int(score_id_by_ix[index].split('_')[0])].last_update.year for index in range(len(wv['vectors']))]
# colors = [min(beatmaps_by_id[int(score_id_by_ix[index].split('_')[0])].total_length, 60*5) for index in range(len(wv['vectors']))]
sizes = [(np.log(1+score_metadata_by_id[score_id_by_ix[index]].num_users)**1.8)*0.05+0.5 for index in range(len(wv['vectors']))]
# alphas = [1 if score_id_by_ix[index] in my_score_ids else 0.1 for index in range(len(wv['vectors']))]
# alphas = [1 if 'DT' in score_id_by_ix[index] else 0 for index in range(len(wv['vectors']))]
alphas = [1 * (1 if score_id_by_ix[index] in my_score_ids else 0.11) if (score_metadata_by_id[score_id_by_ix[index]].avg_pp>10) else 0 for index in range(len(wv['vectors']))]

emb = emblaze.Embedding({Field.POSITION: positions, Field.NAME: names, Field.COLOR: colors, Field.RADIUS: sizes, Field.ALPHA: alphas}, n_neighbors=20,)
emb.compute_neighbors(metric='euclidean')

In [57]:
wv = load_word2vec_format(fname)
emb = emblaze.Embedding({Field.POSITION: positions, Field.NAME: names, Field.COLOR: colors, Field.RADIUS: sizes, Field.ALPHA: alphas}, n_neighbors=20,)
emb.compute_neighbors(metric='euclidean')

umap_params = {
    'densmap': False,
    'dens_lambda': 1,
    # 'min_dist': 0.7,
    'metric': 'euclidean',
}

trimap_params = {
    'n_dims': 2,
    # 'n_inliers': 12,
    'n_outliers': 4,
    # 'weight_temp': 0.5,
}

def trimap_fit_tfn(data, ids, **params):
    del params['metric']
    return trimap.TRIMAP(**params).fit_transform(data)

variants = emblaze.EmbeddingSet([
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=50, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=80, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=200, min_dist=0.3, **umap_params),
    emb.project(method=ProjectionTechnique.UMAP, n_neighbors=150, min_dist=0.25, **umap_params),
    emb.project(method=ProjectionTechnique.UMAP, n_neighbors=170, min_dist=0.25, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=200, min_dist=0.35, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=220, min_dist=0.35, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=250, min_dist=0.35, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=200, min_dist=0.4, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=130, min_dist=0.5, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=130, min_dist=0.6, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=130, min_dist=0.7, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=160, min_dist=0.5, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=160, min_dist=0.6, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=160, min_dist=0.7, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=200, min_dist=0.6, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=200, min_dist=0.7, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=230, min_dist=0.6, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=230, min_dist=0.7, **umap_params),
    # emb.project(method=ProjectionTechnique.UMAP, n_neighbors=200, min_dist=0.99, **umap_params),
])
# variants = emblaze.EmbeddingSet([
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=22, learning_rate='auto', init='pca', early_exaggeration=1.3),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=25, learning_rate='auto', init='pca', early_exaggeration=1.3),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=30, learning_rate='auto', init='pca', early_exaggeration=1.3),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=35, learning_rate='auto', init='pca', early_exaggeration=1.3),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=40, learning_rate='auto', init='pca', early_exaggeration=1.3),
# ])
# variants = emblaze.EmbeddingSet([
#     emb
# ])
# variants = emblaze.EmbeddingSet([
#     emb.project(method=ProjectionTechnique.PCA, n_components=2),
#     emb.project(method=ProjectionTechnique.PCA, n_components=2),
#     emb.project(method=ProjectionTechnique.PCA, n_components=2),
#     emb.project(method=ProjectionTechnique.PCA, n_components=2),
#     emb.project(method=ProjectionTechnique.PCA, n_components=2),
# ])

In [58]:
thumbnails = emblaze.TextThumbnails(names)
for emb in variants.embeddings:
    emb.set_field(Field.NAME, names)
    emb.set_field(Field.COLOR, colors)
    emb.set_field(Field.RADIUS, sizes)
    emb.set_field(Field.ALPHA, alphas)
w = emblaze.Viewer(embeddings=variants, thumbnails=thumbnails)
w.colorScheme = 'turbo'
w

Viewer(colorScheme='turbo', data={'data': [{'_format': 'compressed', '_idtype': 'u2', '_length': 45071, 'ids':…

In [62]:
projected = variants.embeddings[1]
serialized = projected.to_json(compressed=False, save_neighbors=False)

emb_size = len(serialized['points'].keys())

scale = 2.5

# compute center of bounding box and re-center, also apply scale
min_x = min(point['x'] for point in serialized['points'].values())
max_x = max(point['x'] for point in serialized['points'].values())
min_y = min(point['y'] for point in serialized['points'].values())
max_y = max(point['y'] for point in serialized['points'].values())
center_x = (min_x + max_x) / 2
center_y = (min_y + max_y) / 2

transformed_points = []
for i in range(emb_size):
    pt = serialized['points'][i]
    tfn = [
        (pt['x'] - center_x) * scale,
        (pt['y'] - center_y) * scale,
    ]

    tfn = [-tfn[1], -tfn[0]]

    transformed_points.append(tfn)

points = {}
for i in range(emb_size):
    points[score_id_by_ix[i]] = transformed_points[i]

points

import json

with open('../data/embedding_new_2.json', 'wt') as f:
    f.write(json.dumps(points))